In [1]:
import torch
import torchvision
from torch import nn, optim
from art.attacks.evasion import FastGradientMethod
from art.defences.trainer import AdversarialTrainer
from art.estimators.classification import PyTorchClassifier
from sklearn.metrics import accuracy_score

# Load the Fashion-MNIST dataset
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

# Define a simple CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2)(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.LogSoftmax(dim=1)(x)
        return output

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Wrap the trained PyTorch model with ART PyTorchClassifier
classifier = PyTorchClassifier(model=model, loss=criterion, optimizer=optimizer, input_shape=(1, 28, 28), nb_classes=10, clip_values=(0, 1))

# Test the model's accuracy on the original test samples
x_test, y_test = zip(*[(x, y) for x, y in testloader])
x_test, y_test = torch.cat(x_test).numpy(), torch.cat(y_test).numpy()
predictions = classifier.predict(x_test)
accuracy = accuracy_score(y_test, predictions.argmax(axis=1))
print(f"Original test data accuracy:{accuracy * 100:.2f}%")

# Perform an evasion attack (FGSM) on the test samples

attack = FastGradientMethod(estimator=classifier, eps=0.3)
x_test_adv = attack.generate(x=x_test)

# Test the model's accuracy on the adversarial test samples

predictions_adv = classifier.predict(x_test_adv)
accuracy_adv = accuracy_score(y_test, predictions_adv.argmax(axis=1))
print(f"Adversarial test data accuracy: {accuracy_adv * 100:.2f}%")

# Defend the model against the evasion attack using adversarial training
adv_trainer = AdversarialTrainer(classifier, attacks=attack, ratio=0.5)
adv_trainer.fit(x_test, y_test, batch_size=100, nb_epochs=10)

# Retest the model's accuracy on the original test samples after adversarial training

predictions_def = classifier.predict(x_test)
accuracy_def = accuracy_score(y_test, predictions_def.argmax(axis=1))
print(f"Defended test data accuracy: {accuracy_def * 100:.2f}%")

# Retest the model's accuracy on the adversarial test samples after adversarial training
predictions_adv_def = classifier.predict(x_test_adv)
accuracy_adv_def = accuracy_score(y_test, predictions_adv_def.argmax(axis=1))
print(f"Defended adversarial test data accuracy: {accuracy_adv_def * 100:.2f}%")

100%|██████████| 26421880/26421880 [00:07<00:00, 3659756.96it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 504455.46it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3380886.27it/s]


...

Adversarial training epochs:   0%|          | 0/10 [00:00<?, ?it/s]

/home/mrnbd/.local/lib/python3.10/site-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Defended test data accuracy: 94.77%


/home/mrnbd/.local/lib/python3.10/site-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Defended adversarial test data accuracy: 90.29%
